In [1]:
%load_ext watermark
%load_ext autoreload

import os
import warnings
import numpy as np
import xarray as xr
import intake
import intake_esm

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.16.1

conda environment: clisci

sys       : 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
intake_esm: 2023.11.10
intake    : 0.0.0
xarray    : 2023.10.1
numpy     : 1.24.4



# Option 1: use `intake-esm`

This method uses the `intake-esm` package which requires a json catalog file that contains the CMIP output file locations on the NCAR filesystem. The catalog file already exists, but I'm not sure how often it is updated. This method is quick and easy to use but may not always include newly added CMIP output files (likely due to the catalog file not being updated frequently). The `intake.open_esm_datastore` line often takes a while to run but can be sped up with Dask.

The documentation is here: https://intake-esm.readthedocs.io/en/stable/

In [5]:
# Load the catalog file
catalog_file = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json'
cat = intake.open_esm_datastore(catalog_file)

In [20]:
# Take a peek at the underlying dataframe
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370-lowNTCF,r1i1p1f1,day,rsds,gn,NaN,v20190624,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370-lowNTCF,r1i1p1f1,day,tasmax,gn,NaN,v20190624,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370-lowNTCF,r2i1p1f1,day,rsds,gn,NaN,v20190624,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370-lowNTCF,r2i1p1f1,day,tasmax,gn,NaN,v20190624,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370-lowNTCF,r3i1p1f1,day,rsds,gn,NaN,v20190624,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
...,...,...,...,...,...,...,...,...,...,...,...,...
2325954,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,6hrLev,va,gn,NaN,v20190625,209803010000-209803311800,/glade/collections/cmip/CMIP6/gcm/ScenarioMIP/...
2325955,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,6hrLev,va,gn,NaN,v20190625,209804010000-209804301800,/glade/collections/cmip/CMIP6/gcm/ScenarioMIP/...
2325956,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Oday,tos,gn,NaN,v20210329,20150101-20641231,/glade/collections/cmip/CMIP6/gcm/ScenarioMIP/...
2325957,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Oday,tos,gn,NaN,v20210329,20650101-21001231,/glade/collections/cmip/CMIP6/gcm/ScenarioMIP/...


In [15]:
# Explore the dataframe by listing the unique entries for each property (column)
cat.unique()

activity_id            [AerChemMIP, C4MIP, CDRMIP, CFMIP, CMIP, DAMIP...
institution_id         [BCC, CNRM-CERFACS, HAMMOZ-Consortium, MIROC, ...
source_id              [BCC-ESM1, CNRM-ESM2-1, MPI-ESM-1-2-HAM, MIROC...
experiment_id          [ssp370-lowNTCF, ssp370, hist-1950HC, hist-piN...
member_id              [r1i1p1f1, r2i1p1f1, r3i1p1f1, r1i1p1f2, r2i1p...
table_id               [day, Amon, Omon, SImon, AERday, AERmon, AERmo...
variable_id            [rsds, tasmax, hfss, pr, rsus, tas, ts, ua, va...
grid_label             [gn, gr, gr1, gm, gnz, grz, gra, grg, gr2z, gr...
dcpp_init_year         [1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965....
version                [v20190624, v20200219, v20190702, v20190613, v...
time_range             [20150101-20551231, 201501-205512, 201501-2024...
path                   [/glade/collections/cmip/CMIP6/AerChemMIP/BCC/...
derived_variable_id                                                   []
dtype: object

In [17]:
# Explore and only select the first 10 unique models in the dataframe
cat.unique().source_id[:10]

['BCC-ESM1',
 'CNRM-ESM2-1',
 'MPI-ESM-1-2-HAM',
 'MIROC6',
 'MRI-ESM2-0',
 'CESM2-WACCM',
 'CESM2',
 'NorESM2-LM',
 'UKESM1-0-LL',
 'GFDL-ESM4']

In [53]:
# Filter the dataframe by different properties to create a subset of the full catalog. This is where
# you can chose a specific simulation and variable
cat_subset = cat.search(
    source_id='CESM2',
    table_id='Amon',
    variable_id='tas',
    member_id='r1i1p1f1',
    experiment_id='piControl'
)

In [55]:
# This is the intake-esm way of loading the output from the catalog subset, but it doesn't always work for me
# and I haven't spent the time to debug
cat_dict = cat_subset.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)

In [ ]:
# Instead, I just manually grab the paths to the output files and then call xr.open_mfdataset to load the output
# The downside of this method is that you can only load one variable from one simulation at a time

# Create a list of paths from the catalog subset
path_list = cat_subset.df.path.tolist()

# Create a generic path (no specific dates) for the catalog subset
path = '_'.join(path_list[0].split('_')[:-1]) + '_*.nc'

# Open the dataset while suppressing any SerializationWarnings
with warnings.catch_warnings():
      warnings.simplefilter('ignore')
      ds = xr.open_mfdataset(path, use_cftime=True)

This function wraps up all the steps above.

In [46]:
def load_cmip_variable(
    source_id: str,
    experiment_id: str,
    variable_id: str,
    table_id: str,
    member_id: str,
    cat: intake_esm.core.esm_datastore = None,
    time_subset: slice = None) -> xr.Dataset | xr.DataArray:
    '''
    Load CMIP6 dataset from the NCAR filesystem using intake-esm.
    '''

    # Create the CMIP6 catalog if not provided as a parameter
    if cat is None:
        catalog_file = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json'
        cat = intake.open_esm_datastore(catalog_file)

    # Subset the catalog
    cat_subset = cat.search(
        source_id=source_id,
        experiment_id=experiment_id,
        variable_id=variable_id,
        table_id=table_id,
        member_id=member_id,
    )

    # Create a list of paths from the catalog subset
    path_list = cat_subset.df.path.tolist()
    if len(path_list) == 0:
        raise Exception('No files found in the catalog subset.')

    # Create a generic path (no specific dates) for the catalog subset
    path = '_'.join(path_list[0].split('_')[:-1]) + '_*.nc'

    # Open the dataset while suppressing any SerializationWarnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        ds = xr.open_mfdataset(path, use_cftime=True)

    # Select a time range if specified
    if time_subset is not None:
        ds = ds.sel(time=time_subset)

    return ds

Preloading the catalog and adding it as a parameter drastically improves the performance.

In [51]:
%%time
cat_ds = load_cmip_variable(
    source_id='CESM2',
    experiment_id='piControl',
    variable_id='tas',
    table_id='Amon',
    member_id='r1i1p1f1',
    cat=cat,
)

CPU times: user 1.36 s, sys: 176 ms, total: 1.54 s
Wall time: 1.53 s


In [52]:
%%time
cat_ds = load_cmip_variable(
    source_id='CESM2',
    experiment_id='piControl',
    variable_id='tas',
    table_id='Amon',
    member_id='r1i1p1f1',
    # cat=cat,  # load the catalog file in the function
)

CPU times: user 7.86 s, sys: 768 ms, total: 8.63 s
Wall time: 8.63 s


# Option 2: search the NCAR filesystem

In cases where I have requested CMIP output to be added to the NCAR filesystem, the `intake-esm` and catalog method has not always worked. I think this is because the catalog file is not frequently updated. To get around this, I iterate through the directories where CMIP output is stored and search for the output I want. Honestly, this is a bit more work than the `intake-esm` method and requires more specific information for each simulation but I think it's more foolproof.

In [90]:
def grab_latest_version(v1, v2):
    date1 = int(v1[1:])
    date2 = int(v2[1:])

    if date1 > date2:
        return v1
    if date1 < date2:
        return v2
    return v1


def generate_filename(
    variable_id,
    table_id,
    source_id,
    experiment_id,
    member_id,
    grid_label):
    '''
    Generate a filename for CMIP6 output
    '''
    filename = [
        variable_id,
        table_id,
        source_id,
        experiment_id,
        member_id,
        grid_label,
    ]

    if table_id == 'fx':
        filename = '_'.join(filename) + '.nc'
    else:
        filename = '_'.join(filename) + '_*.nc'
    return filename


def generate_path(
    activity_id,
    institution_id,
    source_id,
    experiment_id,
    member_id,
    table_id,
    variable_id,
    grid_label):
    '''
    Generate a path for CMIP6 output
    '''
    fpath = f'/glade/collections/cmip/CMIP6/{activity_id}/{institution_id}/{source_id}/{experiment_id}/{member_id}/{table_id}/{variable_id}/{grid_label}'

    # Make sure to grab the latest version
    latest_version = 'v00000000'
    version_dirs = os.listdir(fpath)
    for vd in version_dirs:
        if vd[0] == 'v':
            latest_version = grab_latest_version(latest_version, vd)
    
    fpath = f'{fpath}/{latest_version}'

    if variable_id in os.listdir(fpath):
        fpath = f'{fpath}/{variable_id}'
    
    return fpath


def grab_cmip_dataset(
    activity_id,
    institution_id,
    source_id,
    experiment_id,
    member_id,
    table_id,
    variable_id,
    grid_label):
    '''
    Load the CMIP6 output dataset
    '''
    fpath = generate_path(
        activity_id,
        institution_id,
        source_id,
        experiment_id,
        member_id,
        table_id,
        variable_id,
        grid_label
    )
    print(fpath)

    fname = generate_filename(
        variable_id,
        table_id,
        source_id,
        experiment_id,
        member_id,
        grid_label
    )
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        ds  = xr.open_mfdataset(f'{fpath}/{fname}', use_cftime=True)

    return ds

In [96]:
grab_cmip_dataset(
    activity_id='CMIP',
    institution_id='NOAA-GFDL',
    source_id='GFDL-ESM4',
    experiment_id='1pctCO2',
    member_id='r1i1p1f1',
    table_id='Amon',
    variable_id='tas',
    grid_label='gr1',
)

/glade/collections/cmip/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1i1p1f1/Amon/tas/gr1/v20180701/tas


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, time: 1800, lon: 288)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 2.0
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) object 0001-01-16 12:00:00 ... 0150-12-16 12:00:00
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1200, 180, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1200, 288, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1, 180, 288), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/46)
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Amon
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    ...                     ...
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    tracking_id:            hdl:21.14100/4d62897b-9155-43f5-b3b7-070d76059276
    variable_id:            tas
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1